In [1]:
import time
import matplotlib.pyplot as plt

import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models

# Load data

In [2]:
BATCH_SIZE = 16

In [3]:
data_dir = './fox-rabbit-racoon-dataset'

train_transforms = transforms.Compose([transforms.RandomRotation(30),
                                       transforms.RandomResizedCrop(224),
                                       transforms.RandomHorizontalFlip(),
                                       transforms.ToTensor(),
                                       transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

valid_transforms = transforms.Compose([transforms.Resize(255),
                                       transforms.CenterCrop(224),
                                       transforms.ToTensor(),
                                       transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

# Pass transforms in here, then run the next cell to see how the transforms look
train_ds = datasets.ImageFolder(data_dir + '/train', transform=train_transforms)
valid_ds = datasets.ImageFolder(data_dir + '/valid', transform=valid_transforms)

train_dl = torch.utils.data.DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True)
valid_dl = torch.utils.data.DataLoader(valid_ds, batch_size=BATCH_SIZE)

In [4]:
images, labels = next(iter(train_dl))
images.shape, labels.shape

(torch.Size([16, 3, 224, 224]), torch.Size([16]))

# Load model

In [5]:
device     = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


In [6]:
model = models.resnet18(pretrained=True)

In [7]:
model.fc

Linear(in_features=512, out_features=1000, bias=True)

In [8]:
model.fc = nn.Linear(in_features=512, out_features=3) # Change last layer for our problem

In [9]:
model = model.to(device) # Move model to GPU

# Train

In [10]:
num_epochs = 25
criterion  = nn.CrossEntropyLoss()
optimizer  = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
scheduler  = optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1) # Decay LR by a factor of 0.1 every 7 epochs


time_start = time.time()

for epoch in range(num_epochs):
    print(f'Epoch {epoch+1}/{num_epochs}')
        
    ######################### TRIAN
    
    model.train() # Set model to training mode
    train_loss     = 0.0
    train_corrects = 0
    
    for inputs, labels in train_dl: # Iterate over the whole train set (batch after batch).
        
        inputs = inputs.to(device)
        labels = labels.to(device)

        outputs = model(inputs)           # 1) Forward: Use the model to compute the output
        loss = criterion(outputs, labels) # 2) Loss: Compare real and predicted
        _, preds = torch.max(outputs, 1)  # OPTIONAL: Compute the most activated class
        loss.backward()                   # 3) Backward: Compute gradients
        optimizer.step()                  # 4) Optimizer step: update model weight & biases)
        optimizer.zero_grad()             # 5) Optimizer: zero the parameter gradients

        # statistics
        train_loss     += loss.item() * inputs.size(0)
        train_corrects += torch.sum(preds == labels.data)
        
    scheduler.step() # Change the lr

    train_loss = train_loss / len(train_ds)
    train_acc  = train_corrects.double() / len(train_ds)
    print(f'Training Loss: {train_loss:.4f} Acc: {train_acc:.4f}')
        
    ######################### VALID
    
    model.eval()   # Set model to evaluate mode        
    valid_loss     = 0.0
    valid_corrects = 0
    
    with torch.no_grad():
        for inputs, labels in valid_dl: # Iterate over the whole train set (batch after batch).
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            loss = criterion(outputs, labels)

            # statistics
            valid_loss += loss.item() * inputs.size(0)
            valid_corrects += torch.sum(preds == labels.data)
    
    valid_loss = valid_loss / len(valid_ds)
    valid_acc  = valid_corrects.double() / len(valid_ds)
    print(f'Validation Loss: {valid_loss:.4f} Acc: {valid_acc:.4f}')
    print()


time_elapsed = time.time() - time_start
print(f'Training complete in {time_elapsed//60:.0f} mins {time_elapsed%60:.0f} secs')

Epoch 1/25
Training Loss: 1.0235 Acc: 0.5220
Validation Loss: 0.5121 Acc: 0.7391

Epoch 2/25
Training Loss: 0.5205 Acc: 0.8187
Validation Loss: 0.1769 Acc: 0.9565

Epoch 3/25
Training Loss: 0.3664 Acc: 0.8846
Validation Loss: 0.1007 Acc: 1.0000

Epoch 4/25
Training Loss: 0.2638 Acc: 0.9066
Validation Loss: 0.0688 Acc: 1.0000

Epoch 5/25
Training Loss: 0.2206 Acc: 0.9286
Validation Loss: 0.0606 Acc: 1.0000

Epoch 6/25
Training Loss: 0.2152 Acc: 0.9176
Validation Loss: 0.0538 Acc: 1.0000

Epoch 7/25
Training Loss: 0.1872 Acc: 0.9505
Validation Loss: 0.0485 Acc: 1.0000

Epoch 8/25
Training Loss: 0.2182 Acc: 0.9066
Validation Loss: 0.0417 Acc: 1.0000

Epoch 9/25
Training Loss: 0.1676 Acc: 0.9341
Validation Loss: 0.0427 Acc: 1.0000

Epoch 10/25
Training Loss: 0.2015 Acc: 0.9396
Validation Loss: 0.0408 Acc: 1.0000

Epoch 11/25
Training Loss: 0.2006 Acc: 0.9451
Validation Loss: 0.0414 Acc: 1.0000

Epoch 12/25
Training Loss: 0.1552 Acc: 0.9451
Validation Loss: 0.0459 Acc: 1.0000

Epoch 13/25
T